# Système multi-agents pour reprise d'installations électiques

L'objectif final est de concevoir une méthodologie pour former une stratégie de réparation du système de distribution HTA en situation de crise.

Techniquement, la conception est divisée en trois parties complémentaires :

    1 - Simuler l'environnement du système de distribution HTA -Anylogic- 
    2 - Sélectionner l'algorithme d'apprentissage par renforcement le plus pertinent pour ce type de problème -PPO- 
    3 - Construire un lien entre le simulateur et l'algorithme d'entraînement RL.

Ensuite, nous lançons l'entraînement du modèle RL sur le simulateur d'environnement et enfin, nous testons notre simulation avec le modèle entraîné.


Remarque :
    
    - Créez un nouveau environnement et installer tous les packages nécessaires
    - N'oubliez pas de changer les chemin d'accès dans le code.

## Implémentation de PPO :

Proximal Policy Optimization (PPO) est utilisée dans plusieurs solutions pour entraîner un modèle de décision sur un environnement industriel. PPO permet d'entraîner un modèle sur un espace d'actions discrètes ou continues. Il est plus rapide à converger et surpasse A2C et DDPG avec de bonnes performances.



Article de "Towards Data Science":
The idea is that PPO improves the stability of the Actor training by limiting the policy update at each training step.


In [1]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import numpy as np
# on utilise Pytorch pour construire et entrainer notre modele PPO
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter
import time

# Les observations sur lesquelles on travaille 

[etatequipement=[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], disponibleequipe=[1, 1, 1], emplequipe=[50, 50, 50], recompns=15500.0]

# Les actions à prendre doivent être sous la forme 

a.req1 = 2  # l'equipe 1 vers l'équipement 1

a.req2 = 6 # l'equipe 2 vers l'équipement 2

a.req3 = 48 # l"equipe 3 vers l'equipement 3

In [2]:
import alpyne
from alpyne.client import alpyne_client
from alpyne.client.alpyne_client import AlpyneClient
from alpyne.data.spaces import Action

    IMPORTANT: Il faut mettre l'emplacement du modèle exporté de Anylogic.
    
    le modèle de simulation doit utiliser la classe RLExperiment pour définir l'espace d'action, d'observation et de configuration initiale.

    Le modèle anylogic doit contenir un événement cyclique (qui se déclenche à chaque période de temps) avec la méthode RLtakeAction().

In [5]:
obs=sim.get_observation()
print(obs)
#print(type(obs))
#éol=obs.etatequipement
#print(éol)
#print(type(equip))


NameError: name 'sim' is not defined

In [3]:
# initialisation d'une couche de Réseau de Neurones - Neural Network 'NN'
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    #Transforme en matrice orthogonale
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

In [4]:
class agent_PPO(nn.Module):
    '''
    Attention ! Il reste des 'len' à modifier par des .size si jamais on les met sous forme de tenseur pytorch
    '''

    def __init__(self,obs0):
       #on considère que obs0 est l'objet alpyne des observations initiales
       #super().__init__ permet d'indiquer que agent_PPO est une sous-classe de nn.Module 
        super(agent_PPO,self).__init__()

        self.obs0=obs0
        self.n_equipe=len(obs0.disponibleequipe)
        self.n_equipement=len(obs0.etatequipement)

        #std = variance
        self.critic = nn.Sequential(
            layer_init(nn.Linear(self.n_equipement, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 1), std=1.0),
            
        )
        # NN de actor (pour la politique)
        self.actor = nn.Sequential(
            layer_init(nn.Linear(self.n_equipement, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, self.n_equipement*self.n_equipe), std=0.01))
    

    def get_value(self, x):
        return self.critic(x)

    def get_action_and_value(self, x, action=None):
        #Les logits sont les probabilités de chaque action 
        logits = self.actor(x)
        if logits.nelement()>self.n_equipe*self.n_equipement:
            logits=logits.reshape([x.size(0),self.n_equipe,self.n_equipement])
        else:
            logits=logits.reshape([self.n_equipe,self.n_equipement])
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(x)



In [64]:
x=torch.tensor([[[1,0.2,3],[2,3,4]],[[3,4,1],[5,6,1]]])
x2=torch.tensor([1,0.2,3,2,3,4,3,4,1,5,6,1])
x3=torch.tensor([0.2,0.4,6])

print(x.size(0))
y=x2.reshape([x.size(0),2,3])
x.nele
print(y)

2
tensor([[[1.0000, 0.2000, 3.0000],
         [2.0000, 3.0000, 4.0000]],

        [[3.0000, 4.0000, 1.0000],
         [5.0000, 6.0000, 1.0000]]])


In [48]:
n_equipe=3
n_equipement=50
logits=torch.rand(n_equipe*n_equipement)
print(logits.size())
logits=logits.reshape([3,50])
print(logits.size())
probs = Categorical(logits=logits)
action = probs.sample()
print(probs.log_prob(action))



torch.Size([150])
torch.Size([3, 50])
tensor([-3.6101, -3.5542, -3.8945])


In [22]:
x=torch.tensor([[1,2],[3,4]])
print(x.sum(dim=-1))

tensor([3, 7])


# Les paramètres 

Paramètres environnement

In [5]:
# seed des expériences
seed =1

# Le nombre des environnements parallèles
num_envs = 1
# Le nombre de pas à exécuter dans un seul environnement pendant la phase policy rollout (128)
num_steps =128

Paramètres d'entrainements 

In [6]:
# Le nombre total des pas-de-temps des expériences
total_timesteps = 500

gamma=0.95 #Discount qui prend en compte jusqu'à 100 étapes dans le calcul de la reward 


# Pour réduire le pas d'apprentissage(learning_rate) des NN, politique(Pi) et valeur (VF) "
anneal_lr = True

learning_rate = 2.5e-4 #Pas d'apprentissage de l'optimizer, on pourra essayer quelque chose de l'ordre de 10^-3 par la suite 

batch_size = int(num_envs * num_steps)
num_minibatches = 4

minibatch_size = int(batch_size // num_minibatches)

epochs=10

epsilon_greedy=0.1 #Probabilité d'explorer initiale 

# Lambda pour GAE
gae_lambda = 0.95

# Utilisation de GAE : General Advantage Estimation 
gae = True

# le seuil de KL divergence
target_kl = None

# permmettre la normalisation de l'advantages (soustraire la moyenne / diviser par l'écart type)
norm_adv = True

# le coefficient de clipping PPO (epsilon)
clip_coef = 0.2
# Permettre l'utilisation de clipped loss dans la fonction valeur
clip_vloss = True

#Coefficient d'entropie. Dans le PPO un coefficient d’entropie est multiplié à l’entropie maximale possible 
# et ajouté à la fonction objectif. Ceci contribue à la régularisation en prévenant une convergence prématurée 
# vers une action de probabilité élevée. Entropie élevée : actions équiprobables

ent_coef = 0.01
# coefficient de la fonction valeur dans la fonction obj
vf_coef = 0.5
# la norme maximale pour le gradient clipping
max_grad_norm = 0.5

# les K epochs pour mise à jour la politique
update_epochs = 4

In [7]:
# Cette instance est créée pour lancer l'environnement de simulation.
# Pour lancer la simulation le modèle de simulation anylogic doit etre fini avec la classe RLExperiment et exporté.
# Il faut mettre l'emplacement du modèle exporté de Anylogic.
client = AlpyneClient(r"/Volumes/Stock/IA/Code/Anylogic/RL_Anylogic/Exported/model.jar", verbose = True, port = 51150)

2022-12-08 09:19:08,510 [alpyne.client.alpyne_client @ 69][    INFO] Starting model in /Volumes/Stock/IA/Code/Anylogic/RL_Anylogic/Exported
2022-12-08 09:19:08,913 [alpyne.client.alpyne_client @ 89][   DEBUG] Executing:
java -cp ../alpyne/resources/*:../alpyne/resources/alpyne_lib/*:*:cache/*:database/*:lib/*:lib/database/*:lib/ecj/*:lib/gis/*:lib/poi/*:lib/sa/*:lib/database/querydsl/*:lib/sa/jackson/*:lib/sa/spark/* com.anylogic.alpyne.AlpyneServer -p 51150 -o /Volumes/Stock/IA/Code/Anylogic/RL_Anylogic -l ALL .

2022-12-08 09:19:08,943 [alpyne.client.alpyne_client @ 107][    INFO] Started app | PID = 1321
2022-12-08 09:19:08,946 [alpyne.client.http_client @ 31][   DEBUG] GET /versions/number/0: None
2022-12-08 09:19:17,771 [alpyne.client.http_client @ 64][   DEBUG] => 200 (OK) [('Content-Type', 'application/json'), ('Connection', 'close'), ('Content-Length', '1763')] {'version': 0, 'experimentTemplate': {'outputs': [{'name': 'statsEqWorking', 'type': 'STATISTICS_CONTINUOUS', 'value':

In [8]:
# initialisation de la configuration du simulateur 
cfg = client.configuration_template
cfg.nbequipe = 3
cfg.engine_seed = 1  

In [9]:
# lancement d'une simulation avec la configuration initiale
sim = client.create_reinforcement_learning(cfg)
sim=sim.run()

2022-12-08 09:19:22,177 [alpyne.client.http_client @ 31][   DEBUG] POST /runs: {'experimentType': <ExperimentType.REINFORCEMENT_LEARNING: 'REINFORCEMENT_LEARNING'>, 'inputs': [nbequipe:INTEGER=3, {START_TIME}:DOUBLE=0 HOUR, {START_DATE}:DATE_TIME=2019-12-31T23:00:00Z, {STOP_TIME}:DOUBLE=inf HOUR, {STOP_DATE}:DATE_TIME=None, {RANDOM_SEED}:LONG=1]}
2022-12-08 09:19:22,507 [alpyne.client.http_client @ 64][   DEBUG] => 201 (Created) [('Location', 'http://127.0.0.1:51150/runs/1'), ('Content-Type', 'application/json'), ('Connection', 'close'), ('Content-Length', '147')] {'status': 'RUNNING', 'id': 1, 'message': '{"reason":null,"next_event_time":"-Infinity","episode_count":1,"step_count":0,"model_time":0.0}'}


In [13]:
obs=sim.get_observation()
agent=agent_PPO(obs)

2022-11-22 16:52:30,302 [alpyne.client.http_client @ 31][   DEBUG] POST /runs/1/rl: {'command': 'OBSERVATION'}
2022-11-22 16:52:30,412 [alpyne.client.http_client @ 64][   DEBUG] => 200 (OK) [('Content-Type', 'application/json'), ('Connection', 'close'), ('Content-Length', '339')] [{'name': 'etatequipement', 'type': 'INTEGER_ARRAY', 'value': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]}, {'name': 'disponibleequipe', 'type': 'INTEGER_ARRAY', 'value': [1, 1, 1]}, {'name': 'emplequipe', 'type': 'INTEGER_ARRAY', 'value': [50, 50, 50]}, {'name': 'recompns', 'type': 'DOUBLE', 'value': 0.0}]


In [ ]:
sim=sim.reset(inputs=[{'name': 'etatequipement', 'type': 'INTEGER_ARRAY', 'value': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]}, {'name': 'disponibleequipe', 'type': 'INTEGER_ARRAY', 'value': [1, 1, 1]}, {'name': 'emplequipe', 'type': 'INTEGER_ARRAY', 'value': [50, 50, 50]}, {'name': 'recompns', 'type': 'DOUBLE', 'value': 0.0}])

In [58]:
print(sim.get_observation())

2022-11-19 18:46:02,517 [alpyne.client.http_client @ 31][   DEBUG] POST /runs/1/rl: {'command': 'OBSERVATION'}
2022-11-19 18:46:02,540 [alpyne.client.http_client @ 64][   DEBUG] => 200 (OK) [('Content-Type', 'application/json'), ('Connection', 'close'), ('Content-Length', '342')] [{'name': 'etatequipement', 'type': 'INTEGER_ARRAY', 'value': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]}, {'name': 'disponibleequipe', 'type': 'INTEGER_ARRAY', 'value': [1, 0, 1]}, {'name': 'emplequipe', 'type': 'INTEGER_ARRAY', 'value': [30, 44, 6]}, {'name': 'recompns', 'type': 'DOUBLE', 'value': -3000.0}]


[etatequipement=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], disponibleequipe=[1, 0, 1], emplequipe=[30, 44, 6], recompns=-3000.0]


In [14]:
global agent
L=torch.Tensor([])
def train():
    # Initialiser l'agent    
    optimizer = torch.optim.Adam(agent.parameters(), lr=learning_rate, eps=1e-5)

    # Initialiser l'environnement
    global sim
    global L
    #sim = sim.reset()

    # Initialisation de l'espace de stockage (buffer) 
    obs = torch.zeros((num_steps, agent.n_equipement,num_envs)) # Une colonne = 1 environnement différent si on veut upgrade 
    actions = torch.zeros((num_steps, agent.n_equipe,num_envs))
    logprobs = torch.zeros((num_steps,agent.n_equipe, num_envs))
    rewards = torch.zeros((num_steps, num_envs))
    dones = torch.zeros((num_steps, num_envs))
    values = torch.zeros((num_steps, num_envs))

    # Commencer l'entrainement
    global_step = 0
    start_time = time.time()
    next_obs = torch.Tensor(sim.get_observation().etatequipement) # APLYNE, attention : fonctionnel pour un seul environnement
    next_obs=next_obs[:,None ] #À modifier si jamais il y a plus d'un env 
    next_done = torch.zeros(num_envs)
    num_updates = total_timesteps // batch_size
    for update in range(1, num_updates + 1):
        # Déminuer le taux d'apprentissage si aneal_lr : vrai
        if anneal_lr:
            frac = 1.0 - (update - 1.0) / num_updates
            lrnow = frac * learning_rate
            optimizer.param_groups[0]["lr"] = lrnow #Pour changer le learning rate de l'optimizer
        #Phase rollout
        for step in range(num_steps):
            global_step+=1*num_envs
            obs[step]=next_obs #Observations sur le nombre total d'étapes (steps) pour chaque environnement, pour l'instant un seul
            dones[step]=next_done
            with torch.no_grad():
                action,log_prob,_,value=agent.get_action_and_value(next_obs.flatten()) #Changer le flatten en reshape pour plusieurs env
                values[step]=value.flatten()
            
            action=action[:,None ] #À modifier si jamais il y a plus d'un env 
            actions[step]=action

            log_prob=log_prob[:,None]
            logprobs[step]=log_prob

            #Execution des actions
            a=client.action_template

            # A CHANGER
            a.req1=action[0].item()
            a.req2=action[1].item()
            a.req3=action[2].item()

            #
            sim.take_action(a)
            observ=sim.get_observation()
            next_obs,reward,done=observ.etatequipement,observ.recompns,sim.is_terminal()

            reward=torch.Tensor([reward]).view(-1)
            reward=reward[:,None]

            rewards[step]=reward
            next_obs,next_done=torch.Tensor(next_obs),torch.Tensor([done])
            next_obs=next_obs[:,None ] #À modifier si jamais il y a plus d'un env


            print(f'\r global_step={global_step}, step={step}, reward={reward}')
        # Calcul de la fonction Advantage
        with torch.no_grad():
            next_value = agent.get_value(next_obs.flatten()).reshape(1, -1)
            if gae:
                advantages = torch.zeros_like(rewards)
                lastgaelam = 0
                for t in reversed(range(num_steps)):
                    if t == num_steps - 1:
                        nextnonterminal = 1.0 - next_done
                        nextvalues = next_value
                    else:
                        nextnonterminal = 1.0 - dones[t + 1]
                        nextvalues = values[t + 1]
                    delta = rewards[t] + gamma * nextvalues * nextnonterminal - values[t]
                    advantages[t] = lastgaelam = delta + gamma * gae_lambda * nextnonterminal * lastgaelam
                returns = advantages + values
            else:
                returns = torch.zeros_like(rewards)
                for t in reversed(range(num_steps)):
                    if t == num_steps - 1:
                        nextnonterminal = 1.0 - next_done
                        next_return = next_value
                    else:
                        nextnonterminal = 1.0 - dones[t + 1]
                        next_return = returns[t + 1]
                    returns[t] = rewards[t] + gamma * nextnonterminal * next_return
                advantages = returns - values

        # le batch : avoir un vecteur de tous les éléments utilisés
        b_obs = obs.reshape((-1,) + (len(observ.etatequipement),))
        b_logprobs = logprobs.reshape((-1,) + (agent.n_equipe,))
        b_actions = actions.reshape((-1,) + (len(observ.disponibleequipe),))
        b_advantages = advantages.reshape(-1)
        b_returns = returns.reshape(-1)
        b_values = values.reshape(-1)
        #print("advantages is",advantages.size())
        #print("returns is:",returns.size())
        #print('b obs size', b_obs.size())

        

        # Optimisation de la fonction valeur et la politique
        b_inds = np.arange(batch_size)
        clipfracs = []
        for epoch in range(update_epochs):
            np.random.shuffle(b_inds)
            # mini-batching
            for start in range(0, batch_size, minibatch_size):
                end = start + minibatch_size
                mb_inds = b_inds[start:end]
                #print("mb_inds size is:",mb_inds.shape)
                #print("L'élément perturbateur size :",b_obs[mb_inds].size())

                _, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds], b_actions.long()[mb_inds])

                #print("new log prob size :",newlogprob.size())
                #print('b_logprobs size :',b_logprobs[mb_inds].size())
                logratio = newlogprob - b_logprobs[mb_inds]
                ratio = logratio.exp()

                with torch.no_grad():
                    # calcul de approx_kl http://joschu.net/blog/kl-approx.html
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio - 1) - logratio).mean()
                    clipfracs += [((ratio - 1.0).abs() > clip_coef).float().mean().item()]
                    ratio=ratio.sum(dim=-1) # À MODIFIER PLUS TARD

                mb_advantages = b_advantages[mb_inds]
                # normalisation de l'avantage 
                if norm_adv:
                    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)
                # Coût associé à la politique
                pg_loss1 = -mb_advantages * ratio
                pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - clip_coef, 1 + clip_coef)
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                # Coût associé à la fonction valeur
                newvalue = newvalue.view(-1)
                if clip_vloss:
                    v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                    v_clipped = b_values[mb_inds] + torch.clamp(
                        newvalue - b_values[mb_inds],
                        -clip_coef,
                        clip_coef,
                    )
                    v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                    v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                    v_loss = 0.5 * v_loss_max.mean()
                else:
                    v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()

                entropy_loss = entropy.mean()
                loss = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
                optimizer.step()
                with torch.no_grad():
                    loss=torch.tensor([loss.item()])
                    L=torch.cat((L,loss))

            # early stopping
            if target_kl is not None:
                if approx_kl > target_kl:
                    break
        
            #print("epoch loss is :",loss)
    y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
    var_y = np.var(y_true)
    end_time=time.time()
    print("computation time :",end_time-start_time)
        
    explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y



In [ ]:
train()

In [16]:
print(L)

tensor([39168536., 32479252., 23933860., 45106832., 29999458., 43155684.,
        40442900., 27088794., 47407168., 39997424., 23265914., 30015300.,
        29008528., 33437918., 42302484., 35936404., 49460996., 43876544.,
        46941744., 44964892., 41507900., 49986580., 49180940., 44565304.,
        51579872., 45576344., 42606384., 45477308., 42414560., 44894228.,
        47081884., 50849228., 62407844., 64817040., 60078184., 71117744.,
        63103188., 68113504., 63325924., 63875972., 58342668., 68347072.,
        66502884., 65223240., 68107104., 63713556., 61031724., 65562824.])


## ALPyne : 

In [38]:
L=torch.Tensor([])
x=torch.tensor([2])
L=torch.cat((L,x))
print(L)


tensor([2.])


ALPyne (AnyLogic-Python connector) est un outil implémenté pour connecter l'environnement du simulateur anylogic avec des algorithmes python qui permettent l'entraînement d'un modèle RL en se basant sur le modèle exporté de Anylogic.

    Anylogic est un outil de simulation pour les environnements industriels, plus d'infos : https://www.anylogic.com/

## Entrainement du modele 

Dans cette partie, nous établissons un lien entre l'environnement Anylogic et l'environnement d'entraînement Python en utilisant la bibliothèque Alpyne.


Le rôle d'Alpyne est de permettre l'utilisation de l'environnement développé sur Anylogic comme environnement virtuel pour entraîner un modèle de décision utilisant un algorithme RL.



Dans ce qui suit, un exemple d'entraînement d'un modèle décisionnel PPO sur un environnement de parc éolien est présenté. Le modèle récupère l'observation (l'état actuel de l'environnement) et prend une action sous forme d'envoi d'équipes de réparation, l'action est transmise et exécutée par l'environnement qui change d'état et renvoie une récompense qui définit la qualité de l'action précédente.



OpenAI développe une bibliothèque de base stable dont le PPO fait partie des algorithmes RL implémentés, le PPO implémenté dans cette bibliothèque est plus optimisé. Le PPO implémenté nécessite l'utilisation d'un environnement compatible avec les normes de la bibliothèque de gymnastique. Afin de répondre à ce besoin, alpyne fournit une classe abstraite "BaseAlpyneEnv" qui permet d'implémenter les fonctions d'un environnement de gym.



Il est conseillé de redémarrer l'environnement d'exécution python lors de l'apprentissage d'un nouveau modèle, car dans de nombreux cas, le serveur créé par Alpyne reste actif après la fin de l'exécution.

In [30]:
### cette fonction retourne une instance de l'environnement gym avec l'id correspondant
"""Cette implmentation pour verifier l'algorithme aec gym  environement"""
def make_env(gym_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(gym_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env.seed=seed
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk

In [33]:
import numpy as np
import gym
# gym pour implémenter l'environnement d'apprentissage
from gym import spaces

# on utilise les blocs Try:Except: afin de supprimer les commentaires d'installation ou avertassiement.

from alpyne.client.alpyne_client import AlpyneClient
from alpyne.client.model_run import ModelRun
from alpyne.data.spaces import Observation, Action

from alpyne.client.abstract import BaseAlpyneEnv

# création de l'environnement d'apprentissage avec standards gym à travers l'héritage et implémentation des
# fonctions de BaseAlpyneEnv

class EolienParc(BaseAlpyneEnv):

    def __init__(self, sim: ModelRun):
        super().__init__(sim)

    # définie l'espace des observation sous forme d'un vecteur avec les valeurs min et max de chaque champs
    def _get_observation_space(self) -> spaces.Space:
        low1 = np.zeros(50)
        high1 = np.ones(50)
        low2 = np.zeros(3)
        high2 = np.ones(3)
        low3 = np.array([0, 0, 0])
        high3 = np.array([50, 50, 50])
        low4 = np.array([-4500])
        high4 = np.array([+4500])
        return spaces.Box(low=np.concatenate((low1, low2, low3, low4)), high=np.concatenate((high1, high2, high3, high4)))

    # transformer l'observation récupérer par anylogic sous forme d'un vecteur de type Space de gym
    def _convert_from_observation(self, observation: Observation):
        print("observation.etatequipement")
        print(observation.etatequipement)
        print("observation.disponibleequipe")
        print(observation.disponibleequipe)
        print("observation.emplequipe")
        print(observation.emplequipe)
        print("observation.recompns")
        print(observation.recompns)
        print("FIN")
        return np.array(np.concatenate((observation.etatequipement, observation.disponibleequipe, observation.emplequipe)))

    # définie l'espace d'action sous forme d'un vecteur avec les valeurs min et max de chaque action
    def _get_action_space(self) -> spaces.Space:
        return spaces.Box(low=np.array([0, 0, 0]), high=np.array([50, 50, 50]), shape=(3,), dtype=np.int64)

    # transformer l'action prise par l'actor sous forme d'un vecteur à une action de type Action de Alpyne pour
    # la communiquer avec le serveur alpyne
    def _convert_to_action(self, action: np.ndarray) -> Action:
        if(action[0] == action[1]):
            action[1] = 50
        if(action[0] == action[2]):
            action[2] = 50
        if(action[1] == action[2]):
            action[2] = 50
        return Action(req1=int(action[0]), req2=int(action[1]), req3=int(action[2]))

    # calcul de la récompense
    def _calc_reward(self, observation: Observation) -> float:
        return observation.recompns

    # définie une état terminale de l'epsode hors un nombre limité de pas
    def _terminal_alternative(self, observation: Observation) -> bool:
        eq = np.array(observation.etatequipement)
        return np.count_nonzero(eq) >= 49
        # arbitrarily chosen small(ish) number


#sim = client.create_reinforcement_learning(cfg)

# utiliser notre implémentation de l'environnement
env = EolienParc(sim)

env._get_observation_space()



ValueError: The provided model run should not have been started!